In [1]:
from datasketch import MinHash,MinHashLSH,MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re
#读取文件
f = open('./weibos.txt','r',encoding='UTF-8')
text = f.read()
#以。！？作为分割并去掉换行符
sentences = re.split(r'[。！？]',text.replace('\n',''))

#print(sentences)
#若最后一行为空去除
if sentences[-1] == '':
    sentences.pop()
#print(sentences)
#print(len(sentences))

#将item_txt分词
#jieba pseg用法和自定义词典的方法https://www.cnblogs.com/OliverQin/p/8983164.html
def get_item_str(item_txt):
    item_str = ''
    item=(pseg.cut(item_txt)) #pseg可以对切割的词汇标注词性
    for i in list(item):
        #去掉停用词,i.flag表示词性，i.word表示单词
        if i.word not in list(stop):
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += ' '
    return item_str
#对item_str 创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

#设置停用词
stop = [line.strip().encode('utf-8') for line in open('stopwords.txt').readlines()]
#得到分词后的documents
documents = []
for item_text in sentences:
    #将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

#创建LSH Forest及MinHash对象
minhash_list= []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i,temp)
#index所有key以便可以进行检索
forest.index()

query = '国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职'
#将item_txt分词
item_str = get_item_str(query)
#得到item_str的MinHash
minhashquery = get_minhash(item_str)

#查询forest中与m1相似的top-k个邻居
result = forest.query(minhashquery,3)
for i in range(len(result)):
    print(result[i],minhashquery.jaccard(minhash_list[result[i]]),documents[result[i]].replace(' ',''))
print('Top 3 邻居',result)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Ricardo\AppData\Local\Temp\jieba.cache
Loading model cost 0.675 seconds.
Prefix dict has been built succesfully.


1 0.5703125 ​国足输给叙利亚之后，里皮辞职
34 0.3515625 国足输给叙利亚后，里皮坐不住了，直接辞职了难怪有网友说，爱护生命，远离男足
37 1.0 国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职
Top 3 邻居 [1, 34, 37]
